In [ ]:
import zipfile

# Путь к файлу архива
zip_file_path = '/mnt/data/sirius_data-6fb0f21e-6f0f-4d7d-9a83-dab650e7ea10.zip'
extracted_folder = '/mnt/data/extracted_sirius_data'

# Распаковка архива
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Проверка содержимого папки
extracted_files = os.listdir(extracted_folder)
extracted_files

In [ ]:
# Проверка содержимого папки 'sirius_data'
sirius_data_folder = os.path.join(extracted_folder, 'sirius_data')
sirius_data_files = os.listdir(sirius_data_folder)
sirius_data_files

In [ ]:
import cv2
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import os

In [ ]:
# Загрузка предобученной модели U2-Net
model = torch.hub.load('Nkap23/U-2-Net', 'u2net')

In [ ]:
# Функция для удаления фона
def remove_background(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize((320, 320)), transforms.ToTensor()])
    image_tensor = transform(image).unsqueeze(0)

    # Получаем маску фона
    with torch.no_grad():
        mask = model(image_tensor)[0][0]

    # Преобразование маски
    mask = mask.squeeze().cpu().numpy()
    mask = cv2.resize(mask, (image.size[0], image.size[1]))
    mask = (mask > 0.5).astype(np.uint8)

    # Применение маски для удаления фона
    image_np = np.array(image)
    result = image_np * np.expand_dims(mask, axis=2)
    return Image.fromarray(result)


In [ ]:
# Функция для замены фона
def replace_background(image, background_color=(255, 255, 255)):
    background = np.full(image.size + (3,), background_color, dtype=np.uint8)
    image_np = np.array(image)
    mask = (image_np[:, :, :3] != 0).any(axis=-1).astype(np.uint8)
    result = image_np.copy()
    result[mask == 0] = background[mask == 0]
    return Image.fromarray(result)

# Обработка всех изображений
image_folder = sirius_data_folder
output_folder = '/mnt/data/processed_images'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_folder, filename)
        result = remove_background(image_path)
        final_image = replace_background(result, background_color=(255, 255, 255))  # Белый фон
        final_image.save(os.path.join(output_folder, filename))

In [ ]:
#генерация описания товара

import random

# Генерация описания
def generate_description(filename, style="concise"):
    item_id = os.path.splitext(filename)[0]

    # Простая генерация описаний
    descriptions = {
        "concise": f"Товар {item_id}: Высококачественный продукт, созданный для вашего комфорта.",
        "detailed": f"Товар {item_id}: Идеальное сочетание стиля и качества. Подходит для использования в повседневной жизни и подчеркивает ваш вкус."
    }

    return descriptions.get(style, descriptions["concise"])

# Пример генерации описаний для всех файлов
for filename in os.listdir(output_folder):
    if filename.endswith('.jpg'):
        description = generate_description(filename, style="concise")
        print(f"Описание для {filename}: {description}")